In [18]:
import pandas as pd
import os
from openpyxl import load_workbook

In [19]:
# Folder where the workbooks are stored
folder_path = '/Users/rajatkumarjena/Downloads/Cyclist'

# List of months for the workbook names
months = [
    'April 2025','May 2025'
]

In [20]:
# Create an empty DataFrame to store the combined data
combined_data = pd.DataFrame()

# Chunk size for reading each file (adjust as needed)
chunk_size = 100000

In [21]:
# Function to read Excel file and process it in chunks
def process_excel_file(file_path, chunk_size):
    wb = load_workbook(file_path, read_only=True)  # Load the workbook in read-only mode
    sheet = wb.active  # Use the active sheet, or you can specify sheet name like wb['Sheet1']
    
    # Read the header row
    columns = [cell.value for cell in sheet[1]]  # First row is the header
    data = []
    
    # Process rows in chunks
    for idx, row in enumerate(sheet.iter_rows(min_row=2, values_only=True), start=2):  # Skip header
        data.append(row)
        
        # When the chunk size is reached, process and reset the chunk
        if idx % chunk_size == 0:
            chunk_df = pd.DataFrame(data, columns=columns)
            yield chunk_df
            data = []  # Reset the chunk
    
    # If there is remaining data after the loop ends
    if data:
        chunk_df = pd.DataFrame(data, columns=columns)
        yield chunk_df

# Loop through each month and process the respective workbook
for month in months:
    file_name = f'{month}.xlsx'  
    file_path = os.path.join(folder_path, file_name)

    # Check if the file exists
    if os.path.exists(file_path):
        print(f"Processing {file_name}...")
        
        # Process the Excel file in chunks
        for chunk in process_excel_file(file_path, chunk_size):
            # Concatenate the current chunk to the combined data
            combined_data = pd.concat([combined_data, chunk], ignore_index=True)
    else:
        print(f"File {file_name} not found, skipping...")


Processing April 2025.xlsx...
Processing May 2025.xlsx...


Exception ignored in: <function ZipFile.__del__ at 0x1065baa20>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/zipfile/__init__.py", line 1940, in __del__
    self.close()
  File "/opt/anaconda3/lib/python3.12/zipfile/__init__.py", line 1957, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file


In [22]:
# Save the combined data into a new Excel file
combined_data.to_excel('combined_data_2024.xlsx', index=False)

print("Data has been successfully combined and saved as 'combined_data_2024.xlsx'")

Data has been successfully combined and saved as 'combined_data_2024.xlsx'
